In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
765,I first saw Love in Limbo playing late on free...,positive
834,One more of those brilliant young men who went...,positive
914,This is probably the best cinematic depiction ...,positive
446,First and foremost this movie has the stupides...,negative
801,"A family with dad Louis (Dale Midkiff), mom Ra...",negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
765,first saw love limbo playing late free air tv ...,positive
834,one brilliant young men went dared make teen r...,positive
914,probably best cinematic depiction life manhatt...,positive
446,first foremost movie stupidest plot movie ever...,negative
801,family dad louis dale midkiff mom rachel denis...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    256
positive    244
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
765,first saw love limbo playing late free air tv ...,1
834,one brilliant young men went dared make teen r...,1
914,probably best cinematic depiction life manhatt...,1
446,first foremost movie stupidest plot movie ever...,0
801,family dad louis dale midkiff mom rachel denis...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/mohammedbismilla76/mlops1.mlflow')
dagshub.init(repo_owner='mohammedbismilla76', repo_name='mlops1', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as mohammedbismilla76

Initialized MLflow to track repo "mohammedbismilla76/mlops1"

Repository mohammedbismilla76/mlops1 initialized!

2025/03/25 15:54:07 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/8ff9ca913f8e4814beb6d21d6da413d3', creation_time=1742898248821, experiment_id='0', last_update_time=1742898248821, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [12]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-25 15:56:18,152 - INFO - Starting MLflow run...
2025-03-25 15:56:19,137 - INFO - Logging preprocessing parameters...
2025-03-25 15:56:20,308 - INFO - Initializing Logistic Regression model...
2025-03-25 15:56:20,310 - INFO - Fitting the model...
2025-03-25 15:56:20,788 - INFO - Model training complete.
2025-03-25 15:56:20,788 - INFO - Logging model parameters...
2025-03-25 15:56:21,169 - INFO - Making predictions...
2025-03-25 15:56:21,173 - INFO - Calculating evaluation metrics...
2025-03-25 15:56:21,220 - INFO - Logging evaluation metrics...
2025-03-25 15:56:22,676 - INFO - Saving and logging the model...
2025/03/25 15:56:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-25 15:56:47,817 - INFO - Model training and logging completed in 28.68 seconds.
2025-03-25 15:56:47,817 - INFO - Accuracy: 0.576
2025-03-25 15:56:47,817 - INFO - Precision: 

🏃 View run fearless-asp-829 at: https://dagshub.com/mohammedbismilla76/mlops1.mlflow/#/experiments/0/runs/9252a48db26e4487a014f110afc3af10
🧪 View experiment at: https://dagshub.com/mohammedbismilla76/mlops1.mlflow/#/experiments/0


In [4]:
import mlflow
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/mohammedbismilla76/mlops1.mlflow')
dagshub.init(repo_owner='mohammedbismilla76', repo_name='mlops1', mlflow=True)

client = mlflow.MlflowClient()

model_name = "my_model"
# Get the latest version in staging
latest_version_staging = client.get_latest_versions(model_name, stages=["staging"])[0].version
print(latest_version_staging)

Initialized MLflow to track repo "mohammedbismilla76/mlops1"

Repository mohammedbismilla76/mlops1 initialized!

C:\Users\dmadmin\AppData\Local\Temp\ipykernel_17940\2899090235.py:10: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version_staging = client.get_latest_versions(model_name, stages=["staging"])[0].version


10


In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
stage = "Staging"

# Get all models in the staging stage
staged_models = client.search_model_versions(f"current_stage='{stage}'")

for model in staged_models:
    print(f"Model: {model.name}, Version: {model.version}")
    print(f"Run ID: {model.run_id}\n")